# Chapter 8 Exercises

1. _What are the main motivations for reducing a dataset’s dimensionality? What are the main drawbacks?_<br>
<br>
You can reduce dimensions to visualize. You can also reduce dimensions to simplify the model needed for the data, or perhaps to speed up training.
However, you may lose prediction power if information is lost<br>
<br>
1. _What is the curse of dimensionality?_<br>
<br>
Higher dimensional spases tend to have sparser datasets, so extrapolations are larger, so models tend to overfit compared to lower dimensality<br><br>
1. _Once a dataset’s dimensionality has been reduced, is it possible to reverse the operation? If so, how? If not, why?_<br>
<br>
It is only possible if the two features were combined in an independent way, such as all possible combinations of two classes<br><br>
1. _Can PCA be used to reduce the dimensionality of a highly nonlinear dataset?_<br>
<br>
<br><br>
1. _Suppose you perform PCA on a 1,000-dimensional dataset, setting the explained variance ratio to 95%. How many dimensions will the resulting dataset have?_<br>
<br>
<br>
<br><br>
1. _In what cases would you use regular PCA, Incremental PCA, Randomized PCA, or Random Projection?_<br>
<br>
<br><br>
1. _How can you evaluate the performance of a dimensionality reduction algorithm on your dataset?_<br>
<br>
compare the predictive performance of models with and without the dim. reduction
<br><br>
1. _Does it make any sense to chain two different dimensionality reduction algorithms?_<br>
<br>
Yes, the dataset may have large scale structure best served by one algorithm, whose output may have structure better served by another algorithm
<br><br>
1. _Load the MNIST dataset (introduced in Chapter 3) and split it into a training set and a test set (take the first 60,000 instances for training, and the remaining 10,000 for testing). Train a Random Forest classifier on the dataset and time how long it takes, then evaluate the resulting model on the test set. Next, use PCA to reduce the dataset’s dimensionality, with an explained variance ratio of 95%. Train a new Random Forest classifier on the reduced dataset and see how long it takes. Was training much faster? Next, evaluate the classifier on the test set. How does it compare to the previous classifier? Try again with an SGDClassifier. How much does PCA help now?_

0.9853954065635522

10. _Use t-SNE to reduce the first 5,000 images of the MNIST dataset down to two dimensions and plot the result using Matplotlib. You can use a scatterplot using 10 different colors to represent each image’s target class. Alternatively, you can replace each dot in the scatterplot with the corresponding instance’s class (a digit from 0 to 9), or even plot scaled-down versions of the digit images themselves (if you plot all digits, the visualization will be too cluttered, so you should either draw a random sample or plot an instance only if no other instance has already been plotted at a close distance). You should get a nice visualization with well-separated clusters of digits. Try using other dimensionality reduction algorithms such as PCA, LLE, or MDS and compare the resulting visualizations._